In [12]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 
import win32com.client


#Importar libs do programa 
import design_functions
from v_p_viga_sobre_pilar.v_p_viga_sobre_pilar import *
from v_p_chapa_cabeca.v_p_chapa_cabeca import parametro_b,espessura_solda
from draw_autocad.draw_autocad_figures import *
from materials import * 

In [13]:
perfil_pilar = W_610x174_0

aço_escolhido = ASTM_A36

aco_chapa = ASTM_A36

solda_escolhida =E70XX

perfil_pilar.inercias()
perfil_pilar.material(aço_escolhido)


##### Cálculo da disposição da chapa, dos parafusos e do diâmetro minimo

In [30]:
M = 6500 #kN.mm 

V = 720 #kN

T = 480 #kN

parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =1)

enrijecedor = 0

S = dim_chapa_pilar(M,V,T,aco_chapa,enrijecedor,perfil_pilar,parafuso,gamma)

In [31]:
S

[0,
     parafuso  x (mm)      y (mm)
 0          1      40   19.000000
 1          2     285   19.000000
 2          3      40  100.600000
 3          4     285  100.600000
 4          5      40  140.046154
 5          6     285  140.046154
 6          7      40  179.492308
 7          8     285  179.492308
 8          9      40  218.938462
 9         10     285  218.938462
 10        11      40  258.384615
 11        12     285  258.384615
 12        13      40  297.830769
 13        14     285  297.830769
 14        15      40  337.276923
 15        16     285  337.276923
 16        17      40  376.723077
 17        18     285  376.723077
 18        19      40  416.169231
 19        20     285  416.169231
 20        21      40  455.615385
 21        22     285  455.615385
 22        23      40  495.061538
 23        24     285  495.061538
 24        25      40  534.507692
 25        26     285  534.507692
 26        27      40  573.953846
 27        28     285  573.953846
 28       

In [32]:
parafuso.diametro_mm

12.7

In [33]:
chapa=S[2]
ver_parafuso=S[3]
exp=S[4]
if enrijecedor ==1:
    enj = S[5]
    print(enj)

In [34]:
exp

9.524999999999999

##### Cálculo da espessura do filete de solda na alma e na mesa

In [29]:
espessura = espessura_solda(M,T,V,solda_escolhida,perfil_pilar,exp,True,gamma)

esp = math.ceil(espessura) #mm

esp

5

##### Abertura do projeto no pyautocad

In [63]:
# Cria instância do AutoCAD
acad = win32com.client.Dispatch("AutoCAD.Application")
acad.Visible = True  # Garante que a janela fique visível

In [64]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing1.dwg


In [65]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


In [66]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

In [67]:
# Chamando a função para desenhar a chapa 3D
objetos_chapa = criar_chapa_3d(acad, chapa.df, exp)

In [68]:
objetos_parafusos=[]
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]

    # Adicionar circunferência no ponto
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,exp), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,0), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    # Transladar hexágono para o ponto atual
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, y_centro, exp)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(*hexagono_transladado[j])
        p2 = APoint(*hexagono_transladado[j + 1])
        obj = acad.model.AddLine(p1, p2)
        objetos_parafusos.append(obj)

In [69]:
base_perfil= min(ver_parafuso['y (mm)'])+ parametro_b(parafuso.diametro_mm)

In [70]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_pilar, (chapa.B / 2) - (perfil_pilar.b_f / 2), posicao_y=base_perfil, altura_z=exp)

In [71]:
if enrijecedor == 1:
    desenhar_enrijecedores(acad, (0,0,exp),base_perfil ,chapa, perfil_pilar, ver_parafuso, parafuso.diametro_mm, enj)